In [3]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import cyipopt
from pybullet_suite import *

In [4]:
from kinjax import kinjax
from jaxlie import *

In [5]:
num_steps = 1
# p_obs = jnp.array([0, 0.1])
# r_obs = 0.2

dim_robot = 7
#r_robot = 0.1
#safe_dist = 0.05

In [6]:
bw = BulletWorld(gui=True)
sm = BulletSceneMaker(bw)
pandas = [bw.load_robot(Panda, f'panda{i}') for i in range(num_steps)]

argv[0]=
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = 
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3090/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 515.105.01
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 515.105.01
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3090/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [7]:
panda = pandas[0]
q_init = jnp.array([0.,0.,0.,-1.5708,0.,1.8675, 0.])
q_goal = panda.get_random_config()
pose_goal = panda.forward_kinematics(q_goal)
T_goal = jnp.array(pose_goal.as_matrix())

In [27]:
# util functions
def to_mat(x):
    return x.reshape(-1, dim_robot)
def at_timestep(i, x):
    return to_mat(x)[i]
def to_v(x):
    return (to_mat(x)[1:] - to_mat(x)[:-1]).flatten()
def Tmat_to_Tvec(T):
    R, t = T[:3, :3], T[:3, -1]
    return jnp.hstack([t, R[:3, 0], R[:3, 1]])
def rotvec_x_sq(Tvec):
    return Tvec[3:6] @ Tvec[3:6]
def rotvec_y_sq(Tvec):
    return Tvec[6:] @ Tvec[6:]

In [28]:
# functions
link_dict, joint_dict = kinjax.get_link_joint_dict(PANDA_URDF)
fk = kinjax.get_FK_fn(link_dict, joint_dict, 7, "panda_grasptarget")
#fk_vec = lambda x: Tmat_to_Tvec(fk(x))
fk_vec = lambda x: SE3.from_matrix(fk(x)).parameters()
#qtn_mag = lambda x: jnp.dot(fk_vec(x)[:4], fk_vec(x)[:4])
#rotvec_x_mag = lambda x: rotvec_x_sq(Tmat_to_Tvec(fk(x)))
#rotvec_y_mag = lambda x: rotvec_y_sq(Tmat_to_Tvec(fk(x)))

Robot name: panda


In [29]:
g, cl, cu = [], [], []
f = lambda x:jnp.array(0.)
g.append(fk_vec)
cl.extend(SE3.from_matrix(T_goal).parameters())
cu.extend(SE3.from_matrix(T_goal).parameters())

# g.append(rotvec_x_mag)
# cl += [1.]
# cu += [1.]
# g.append(rotvec_y_mag)
# cl += [1.]
# cu += [1.]

In [33]:
lb = jnp.hstack([panda.joint_lower_limit] * num_steps)
ub = jnp.hstack([panda.joint_upper_limit] * num_steps)
cl = jnp.array(cl)
cu = jnp.array(cu)

dim = dim_robot*num_steps
const_dim = len(cl)
_g = lambda x: jnp.hstack([fn(x) for fn in g])

In [34]:
x_rand = jnp.array(np.random.random(dim))
lambda_rand = jnp.array(np.random.random(const_dim))

fns = {}
fns['objective'] = f
fns['constraints'] = _g
fns['gradient'] = jax.grad(f)
fns['const_jacobian'] = jax.jacrev(_g)
fns['obj_hessian'] = jax.hessian(f)
fns['const_hessian'] = jax.hessian(_g)

def _hessian(x, lagrange, obj_factor):
    H = obj_factor * fns['obj_hessian'](x)
    H += jnp.einsum('i,ijk->jk', lagrange, fns['const_hessian'](x))
    return H
jac_row, jac_col = jnp.nonzero(fns['const_jacobian'](x_rand))
fns['jacobianstructure'] =  lambda : (jac_row, jac_col)
hess_row, hess_col = jnp.nonzero(jnp.tril(_hessian(x_rand, lambda_rand, 1.)))
fns['hessianstructure'] =  lambda : (hess_row, hess_col)
def jacobian(x):
    J = fns['const_jacobian'](x)
    row, col = fns['jacobianstructure']()
    return J[row, col]
def hessian(x, lagrange, obj_factor):
    H = _hessian(x, lagrange, obj_factor)
    row, col = fns['hessianstructure']()
    return H[row, col]
fns['jacobian'] = jacobian
fns['hessian'] = hessian 

In [35]:
class Prob:
    pass
prob = Prob()

for name, fn in fns.items():
    if name in ['objective', 'constraints', 'gradient', 'jacobian']:
        setattr(prob, name, jax.jit(fn).lower(x_rand).compile())
    elif 'structure' in name:
        setattr(prob, name, jax.jit(fn).lower().compile())
    elif name in ['hessian']:
        hess_fn = jax.jit(fn).lower(x_rand, lambda_rand, 1.).compile()
        setattr(prob, name, hess_fn)

solver = cyipopt.Problem(
    n=dim, m=const_dim,
    problem_obj=prob,
    lb=lb, ub=ub, cl=cl, cu=cu
)
solver.add_option('tol', 0.001)
#solver.add_option('hessian_approximation', 'limited-memory')

In [37]:
x0 = jnp.hstack([panda.get_random_config()]*num_steps)
xsol, info = solver.solve(x0)
#normal : 0.046

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       45
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        7
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 6.06e-01 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [41]:
sm.view_frame(Pose.from_matrix(np.array(T_goal)))

In [43]:
panda.set_joint_angles(xsol)